In [1]:
# ppf of cauchy
x = qcauchy(runif(10000000),loc=5,scale=2)
1/mean(dnorm(x,mean=5.387142857142857,sd=sqrt(9/7)))

[1] 7.84634

In [2]:
# ppf of norm
x = qnorm(runif(10000000),mean=5.387142857142857,sd=sqrt(9/7))
1/mean(dcauchy(x,loc=5,scale=2))

[1] 7.846282

In [1]:
set.seed(10030607)

# 先写一个函数实现随机朝四个方向走
go = function(x){
  out = sample(c(1,2,3,4),1)
  if (out == 1){
    x[1] = x[1] + 1
  }else if(out == 2){
    x[1] = x[1] - 1
  }else if(out == 3){
    x[2] = x[2] + 1
  }else if(out == 4){
    x[2] = x[2] - 1
  }
  return(x)
}

# 再写f的函数
f = function(x){
  end_point = x[dim(x)[1],]
  D = abs(end_point[1]) + abs(end_point[2])
  R = dim(x[which((x[,1] == end_point[1,1])&(x[,2] == end_point[1,2])),])[1]
  result = exp(-D-R/2)
  return(result)
}

# 设定初始值
n_sam = 5000
tmax = 30
X = as.data.frame(matrix(0,(tmax+1),2*n_sam))
X[1,] = rep(0,2*n_sam)
w = matrix(0,(tmax+1),n_sam)
w[1,] = rep(1/n_sam,n_sam)
ess = rep(0,tmax)
rejuvs = 0

start = Sys.time()
for (i in 1:30) {
  for (j in 1:n_sam) {
    X[(i+1),(2*j-1):(2*j)] = go(X[i,(2*j-1):(2*j)])
    w[(i+1),j] = w[i,j]*f(X[1:(i+1),(2*j-1):(2*j)])[1,1]/f(X[1:i,(2*j-1):(2*j)])[1,1]
  }
  w[(i+1),] = w[(i+1),]/sum(w[(i+1),])
  ess[i] = n_sam/(1+(sd(w[(i+1),])/mean(w[(i+1),]))^2)
  if ((ess[i] <= n_sam/2)&(i != 30)) {
    index_rejuv = sample(1:n_sam,n_sam,prob=w[(i+1),],replace=T)
    X_new = as.data.frame(matrix(0,(tmax+1),2*n_sam))
    for (k in 1:n_sam) {
      X_new[,(2*k-1):(2*k)] = X[,(2*index_rejuv[k]-1):(2*index_rejuv[k])]
    }
    X = X_new
    rejuvs=rejuvs+1
    w[(i+1),]=rep(1/n_sam,n_sam)
  }
}
end = Sys.time()
print(end-start)

Time difference of 10.53468 mins


In [2]:
D = matrix(0,1,n_sam)
M = matrix(0,1,n_sam)
for (i in 1:n_sam) {
  path_temp = X[,(2*i-1):(2*i)]
  D[i] = abs(path_temp[(tmax+1),1]) + abs(path_temp[(tmax+1),2])
  count_M = matrix(0,1,(tmax+1))
  for (j in 1: (tmax+1)) {
    count_M[j] = length(which((path_temp[,1] == path_temp[j,1])&(path_temp[,2] == path_temp[j,2])))
  }
  M[i] = max(count_M)
}

mean_d = weighted.mean(D,w[(tmax+1),])
sd_d = sqrt(sum(D^2*w[(tmax+1),]) - (mean_d)^2)
mean_m = weighted.mean(M,w[(tmax+1),])
sd_m = sqrt(sum(M^2*w[(tmax+1),]) - (mean_m)^2)

In [9]:
sd_d

[1] 1.478701

In [3]:
mean_d

[1] 1.784194

In [4]:
sd_d

[1] 1.478701

In [5]:
mean_m

[1] 4.523362

In [6]:
sd_m

[1] 1.230077